# 模型的假设部分之各类阿片合成物溯源
基本假设：
- 阿片类药物成瘾分布趋势随时间变化。做预测时，我们要考虑时间。最新的趋势将有更高的影响因素。
- 阿片类药物滥用的传播取决于图形位置。我们假定毒品的传播主要是从一个地方到邻近的地方，不会跨越很远的距离。
- 当处理数据丢失时，我们将其视为平均值。

回到我们的问题：
> 描述报告的合成类阿片和海洛因事件（案例）在五个州及其县之间**随时间的传播和特点**。使用您的模型，确定在五个州中的每个州开始使用特定阿片类药物的任何可能位置。如果你的团队确定的模式和特征会继续发展下去，美国政府应该特别关注哪些具体问题?在什么药物识别阈值水平会发生这些问题？你的模型预测它们将在何时何地发生？

既然是随时间的传播与特点，则我们需要考虑时间维度对其进行分析

![](https://s2.ax1x.com/2020/02/09/1faYwR.png)
这是该队的工作流程图例，将其翻译成中文
![](https://s2.ax1x.com/2020/02/09/1fOyuT.jpg)
## PART3 Model
该团队用**推荐系统**的方式来进行毒品传播的预测。其模型反映了“推荐系统”的主要思想，即寻找**相似的区域**。这些相似区域将在很大程度上与报道的合成阿片和海洛因事件的传播和特征相似。这些相似的区域帮助我们扩大传播和特征，并使预测更准确。因而拿到地理位置之后该队首先进行的是similarity calculation
### 基本模型语言描述
我们有100个zone和69中不同的阿片类药物，所以我们通过一个$100×69$的矩阵来描述阿片的传播，每一行代表一个区域编号，每一列代表一个阿片类药物编号。那么这个矩阵中的cell(i, j)将代表i区中j号药物的一些信息。

如何判断两个区域的相似性与模型的性能密切相关。在我们的模型中，我们仔细考虑了三个主要因素。(1)随时间的变化趋势(2)地理因素(3)药物鉴定规模。
- 时间：矩阵中每一个元素为一个列表，包含了6年间该zone该类合成物的数据，在计算任意zone之间的相似性时，我们先找出每种药物的相关性，然后进行加权求和。
- 地理位置:地理因素在药物传播中起着至关重要的作用。人们倾向于从一个区域迁移到相邻的区域。这个过程可能会引发药物传播，从而导致药物一个区域的识别情况更有可能与相邻区域的识别情况相似。**很容易根据任何区域对的经纬度计算它们之间的距离**。根据我们的试验表明，控制距离在200公里以内，模型性能良好。
- 药物规模：药物鉴定的大小是另一个需要考虑的重要因素。与药物识别病例较多的区域相比，药物识别病例较少的区域趋于稳定，但生长或下降有发生突变的趋势。在计算相似度时，我们将所有6年的数据相加，并调用时间衰减因子。
$$Total=\sum_{i=2010}^{2016}data_{i}v^{i-1}$$
将$v=0.7$，这样，年份越近的数据的权重越大。

对于每个区域，我们使用因子(2)找到“close zone”，并根据因子(1)和因子(3)为每个close zone打分。通过排序，
### 在网络中“随机游走”寻找起源点
在这一部分，我们主要想找到一些阿片类药物的来源。假设我们正在浏览网页，我们将打开一个网址，浏览网页，然后根据网页上的URL选择喜欢的子网页。我们将继续沿着网页的链接打开新的网页。阿片类药物的传播也有类似的行为。阿片类药物在城市很受欢迎，并慢慢影响周围的城市。然而，由于城市的不同特点，特定阿片类药物在周边城市的传播速度会有很大的差异。**这里我们假设具有相似特征的城市在传播阿片类药物时也会有相似的模式**。因此，阿片类药物扩散到邻近城市的概率是不同的。这一概率由上述各部分城市之间的相似性来描述。

1. **建立阿片类药物的传播图模型**。假设我们正在寻找海洛因的来源。本模型的第一步是建立阿片类药物的传播映射模型。我们选择了任一年的海洛因数据。图中每个节点代表一个聚类点，每个节点的值代表在该点吸食海洛因的人数。当节点A ($W_A$)的数目大于另一个节点B ($W_B$)时，就会产生从A到B的有向边(A→B)。边的权值为A和B $(sim(A,B) = p_{A→B})$。海洛因生成的图表如下所示。

![](https://s2.ax1x.com/2020/02/09/1hS8vn.png)
（笔记：这个队的图做的是真漂亮啊，跟晶体一样）
2. **根据上面的图模型找到可能成为药物来源的节点**。我们称之为候选集。$CandidateSet =\{node1,node2,node3,...\}$在图论中，一个图的顶点的度是与顶点相关联的边的数量，循环计数两次。在这里，我们将定义更改为从顶点散射的边数（笔记：应该意思就是出度，该结点出度越多越有可能成为传播源）。对于每个结点，我们计算其度$(Deg(v))$，我们选择最大的5个结点并将它们存储在候选集中。
3. **利用随机游走的模拟方法模拟药物的传播行为**。分别模拟候选集合中的结点，每个结点模拟100次，记录其他节点的访问次数($n_i$)，计算每个节点的得分，计算公式如下:最高分是我们要找的来源。

${Score}_i=\sum_{j=0 and j\neq i}^{99}n_j*p_{i→j}$，$(i\in Candidate_set)$
$Source=index(max(Score)|Score\in\{{Score}_1,{Score}_2,{Score}_3,...\}$

In [1]:
import xlrd
import pandas as pd
import numpy as np
from math import *
import xlwt
import operator
import copy
from functools import reduce
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
from itertools import combinations
#from igraph import *
import seaborn as sns # 推荐用seaborn来画图
import pylab
import random
#sns.set_style(’darkgrid’)

In [2]:
#以2010年海洛因为例
excel_2010=pd.ExcelFile('./dataprocess/data_2010_zone.xlsx')# 原始数据
data_2010_zone=pd.read_excel(excel_2010,'Sheet1')

In [3]:
H=nx.DiGraph()

In [7]:
H.add_nodes_from(np.arange(100))

In [18]:
H.nodes[1]

{'drugReprot': 'Codeine'}

In [16]:
for node_i in range(100):
    H.nodes[node_i]['drugReprot']=data_2010_zone.iloc[69*node_i+11,3]

In [20]:
node_i=0
for node_i in range(100):
    for node_j in range(100):
        if H.nodes[node_i]['drugReprot'] > H.nodes[node_j]['drugReprot']:
            H.add_edge(node_i,node_j)

### 模型结论
结果分析，上图显示了问题中出现的所有阿片类药物的来源。由此我们可以分析许多有趣的现象：
- 交通因素:我们可以看到很多阿片类药物的来源，在海洋附近，或五大湖附近。靠近海洋使从国外进口非法药物或阿片类药物变得更容易，使这些地区成为许多阿片类药物的来源。
- 法律因素:我们可以看到俄亥俄州是许多毒品的产地。造成这种情况的原因可能与地方法律法规仍然存在漏洞和政府的监管有关